# Genotype-Phenotype Analysis for Kleefstra Syndrome (OMIM #610253)

In this notebook, we will conduct a genotype-phenotype analysis for **Kleefstra Syndrome (OMIM #610253)**, a rare developmental disorder caused by pathogenic variants in the *EHMT1* gene. This syndrome is characterized by a range of symptoms including severe mental retardation, hypotonia, brachy(micro)cephaly, epileptic seizures, flat face with hypertelorism, synophrys, anteverted nares, everted lower lip, carp mouth with macroglossia, and heart defects. 

First, do the first preprocessing steps, loading all the data, validating it, and generating a nice figure for the variants. This will help us to generate hypotheses about possible genotype/phenotype correlations we would like to investigate. 

In [1]:
import hpotk
store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2024-07-01')

cohort_name = 'EHMT1'
tx_id = 'NM_024757.5'
px_id = 'NP_079033.4'

from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store('0.1.18') as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(cohort_name))
len(phenopackets)

from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets
cohort_creator = configure_caching_cohort_creator(hpo)
cohort, validation = load_phenopackets(
    phenopackets=phenopackets,
    cohort_creator=cohort_creator,
)
validation.summarize()

from gpsea.view import CohortViewable
viewer = CohortViewable(hpo)
report = viewer.process(cohort=cohort, transcript_id=tx_id)
with open('ehmt1_cohort_info.html', 'w') as fh:
    _ = fh.write(report)

from gpsea.model.genome import GRCh38
from gpsea.preprocessing import configure_default_protein_metadata_service, VVMultiCoordinateService
txc_service = VVMultiCoordinateService(genome_build=GRCh38)
pms = configure_default_protein_metadata_service()
tx_coordinates = txc_service.fetch(tx_id)
protein_meta = pms.annotate(px_id)

from gpsea.view import ProteinVisualizer
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15, 8))
visualizer = ProteinVisualizer()
visualizer.draw_protein_diagram(
    tx_coordinates,
    protein_meta,
    cohort,
    ax=ax,
)
fig.tight_layout()
fig.savefig('ehmt1_protein_diagram.png')
plt.show()

from gpsea.view import CohortVariantViewer
viewer = CohortVariantViewer(tx_id=tx_id)
report = viewer.process(cohort=cohort)
with open('ehmt1_all_variants.html', 'w') as fh:
    _ = fh.write(report)

Individuals Processed:   8%|▊         | 10/125 [00:39<06:46,  3.54s/individuals]

Expected a result but got an Error for variant: 9_137710964_137710964_C_--107177bp--
<html><head><title>Error</title></head><body><h2>Request too large</h2></body></html>


Individuals Processed:  26%|██▌       | 32/125 [01:26<03:39,  2.36s/individuals]

Expected a result but got an Error for variant: 9_137762821_137775108_--12288bp--_--528bp--
{"error":"Allele must be A,T,G,C or SO term [got: NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN]"}


Individuals Processed: 100%|██████████| 125/125 [04:19<00:00,  2.08s/individuals]
Validated under permissive policy
Showing errors and warnings
Phenopackets
  patient #10
    variants
     errors:

      #0
       warnings:
       ·Patient PMID_39013458_individual_VUSM110048 has an error with variant 9_137710964_137710964_C_--107177bp--. Try again or remove variant form testing... Expected a result but got an Error. See log for details.
  patient #23
    variants
     errors:

      #0
       warnings:
       ·Patient PMID_39013458_individual_KS13004 has an error with variant SO:1000044_HGNC:24650_EHMT1. Try again or remove variant form testing... Unsupported variant class VariantClass.TRANSLOCATION
  patient #32
    variants
     errors:

      #0
       warnings:
       ·Patient PMID_39013458_individual_Diagnostic110005 has an error with variant 9_137762821_137775108_--12288bp--_--528bp--. Try again or remove variant form testing... Expected a result but got an Error. See log for det

TypeError: write() argument must be str, not HtmlGpseaReport

It seems like we have quite some missense variants. Lets test those against the rest of the cohort!

In [4]:
from gpsea.model import VariantEffect
from gpsea.analysis.predicate.genotype import VariantPredicates, monoallelic_predicate
gt_predicate = monoallelic_predicate(
    a_predicate=VariantPredicates.variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id),
    b_predicate=~VariantPredicates.variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id),
    names=('Missense', 'Rest of the cohort')
)
gt_predicate.display_question()

from gpsea.analysis.predicate.phenotype import prepare_predicates_for_terms_of_interest
pheno_predicates = prepare_predicates_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
    min_n_of_patients_with_term=2,
)

from gpsea.analysis.mtc_filter import HpoMtcFilter
mtc_filter = HpoMtcFilter.default_filter(hpo, term_frequency_threshold=0.2)
mtc_correction = 'fdr_bh'
mtc_alpha = 0.05

from gpsea.analysis.pcats.stats import FisherExactTest
count_statistic = FisherExactTest()

from gpsea.analysis.pcats import HpoTermAnalysis
analysis = HpoTermAnalysis(
    count_statistic=count_statistic,
    mtc_filter=mtc_filter,
    mtc_correction=mtc_correction,
    mtc_alpha=mtc_alpha,
)

result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=gt_predicate,
    pheno_predicates=pheno_predicates,
)
result.total_tests

len(result.phenotypes)

from gpsea.view import MtcStatsViewer
mtc_viewer = MtcStatsViewer()
mtc_report = mtc_viewer.process(result)
mtc_report

from gpsea.view import summarize_hpo_analysis
summary_df = summarize_hpo_analysis(hpo, result)
summary_df

Allele group                            Missense         Rest of the cohort  \
                                           Count Percent              Count   
Feeding difficulties [HP:0011968]           5/11     45%              35/36   
Recurrent infections [HP:0002719]           9/12     75%              48/49   
Hearing impairment [HP:0000365]             4/23     17%              31/71   
Gastroesophageal reflux [HP:0002020]        2/24      8%              26/87   
Chronic constipation [HP:0012450]           9/26     35%              50/90   
Microcephaly [HP:0000252]                   3/25     12%              22/74   
Large for gestational age [HP:0001520]      7/22     32%              11/66   
Aggressive behavior [HP:0000718]            3/22     14%              23/79   
Autism [HP:0000717]                         9/22     41%              44/79   
Intellectual disability [HP:0001249]       24/28     86%              87/94   
Overweight [HP:0025502]                    10/23     43%              28/83   
Developmental regression [HP:0002376]       6/28     21%              29/94   
Atrial septal defect [HP:0001631]            3/3    100%              18/25   
Anxiety [HP:0000739]                        7/22     32%              19/79   
Global developmental delay [HP:0001263]    27/28     96%              86/93   
Joint hypermobility [HP:0001382]            6/28     21%              23/94   
Pes planus [HP:0001763]                     6/28     21%              23/94   
Hypotonia [HP:0001252]                     10/28     36%              37/94   
Strabismus [HP:0000486]                     5/15     33%              19/59   

Allele group                                                                  
                                        Percent Corrected p values  p values  
Feeding difficulties [HP:0011968]           97%           0.005124  0.000270  
Recurrent infections [HP:0002719]           98%           0.164747  0.021606  
Hearing impairment [HP:0000365]             44%           0.164747  0.026860  
Gastroesophageal reflux [HP:0002020]        30%           0.164747  0.034684  
Chronic constipation [HP:0012450]           56%           0.288120  0.075821  
Microcephaly [HP:0000252]                   30%           0.348448  0.110036  
Large for gestational age [HP:0001520]      17%           0.376614  0.138753  
Aggressive behavior [HP:0000718]            29%           0.417631  0.175845  
Autism [HP:0000717]                         56%           0.502209  0.237889  
Intellectual disability [HP:0001249]        93%           0.519089  0.273205  
Overweight [HP:0025502]                     34%           0.753024  0.463158  
Developmental regression [HP:0002376]       31%           0.753024  0.475594  
Atrial septal defect [HP:0001631]           72%           0.789550  0.551282  
Anxiety [HP:0000739]                        24%           0.789550  0.581774  
Global developmental delay [HP:0001263]     92%           0.861039  0.679768  
Joint hypermobility [HP:0001382]            24%           0.872406  0.806256  
Pes planus [HP:0001763]                     24%           0.872406  0.806256  
Hypotonia [HP:0001252]                      39%           0.872406  0.826489  
Strabismus [HP:0000486]                     32%           1.000000  1.000000

In [5]:
summary_df

Allele group                            Missense         Rest of the cohort  \
                                           Count Percent              Count   
Feeding difficulties [HP:0011968]           5/11     45%              35/36   
Recurrent infections [HP:0002719]           9/12     75%              48/49   
Hearing impairment [HP:0000365]             4/23     17%              31/71   
Gastroesophageal reflux [HP:0002020]        2/24      8%              26/87   
Chronic constipation [HP:0012450]           9/26     35%              50/90   
Microcephaly [HP:0000252]                   3/25     12%              22/74   
Large for gestational age [HP:0001520]      7/22     32%              11/66   
Aggressive behavior [HP:0000718]            3/22     14%              23/79   
Autism [HP:0000717]                         9/22     41%              44/79   
Intellectual disability [HP:0001249]       24/28     86%              87/94   
Overweight [HP:0025502]                    10/23     43%              28/83   
Developmental regression [HP:0002376]       6/28     21%              29/94   
Atrial septal defect [HP:0001631]            3/3    100%              18/25   
Anxiety [HP:0000739]                        7/22     32%              19/79   
Global developmental delay [HP:0001263]    27/28     96%              86/93   
Joint hypermobility [HP:0001382]            6/28     21%              23/94   
Pes planus [HP:0001763]                     6/28     21%              23/94   
Hypotonia [HP:0001252]                     10/28     36%              37/94   
Strabismus [HP:0000486]                     5/15     33%              19/59   

Allele group                                                                  
                                        Percent Corrected p values  p values  
Feeding difficulties [HP:0011968]           97%           0.005124  0.000270  
Recurrent infections [HP:0002719]           98%           0.164747  0.021606  
Hearing impairment [HP:0000365]             44%           0.164747  0.026860  
Gastroesophageal reflux [HP:0002020]        30%           0.164747  0.034684  
Chronic constipation [HP:0012450]           56%           0.288120  0.075821  
Microcephaly [HP:0000252]                   30%           0.348448  0.110036  
Large for gestational age [HP:0001520]      17%           0.376614  0.138753  
Aggressive behavior [HP:0000718]            29%           0.417631  0.175845  
Autism [HP:0000717]                         56%           0.502209  0.237889  
Intellectual disability [HP:0001249]        93%           0.519089  0.273205  
Overweight [HP:0025502]                     34%           0.753024  0.463158  
Developmental regression [HP:0002376]       31%           0.753024  0.475594  
Atrial septal defect [HP:0001631]           72%           0.789550  0.551282  
Anxiety [HP:0000739]                        24%           0.789550  0.581774  
Global developmental delay [HP:0001263]     92%           0.861039  0.679768  
Joint hypermobility [HP:0001382]            24%           0.872406  0.806256  
Pes planus [HP:0001763]                     24%           0.872406  0.806256  
Hypotonia [HP:0001252]                      39%           0.872406  0.826489  
Strabismus [HP:0000486]                     32%           1.000000  1.000000

In [6]:
print(summary_df[summary_df[('', 'Corrected p values')] < 0.05])

Allele group                      Missense         Rest of the cohort          \
                                     Count Percent              Count Percent   
Feeding difficulties [HP:0011968]     5/11     45%              35/36     97%   

Allele group                                                   
                                  Corrected p values p values  
Feeding difficulties [HP:0011968]           0.005124  0.00027  


Lets also try to investigate a possible sex difference, since this has been previously mentioned in Kleefstra. 

In [7]:
from gpsea.analysis.predicate.genotype import sex_predicate
gt_predicate = sex_predicate()
gt_predicate.display_question()

'Sex of the individual: FEMALE, MALE'

In [9]:
from gpsea.analysis.predicate.phenotype import prepare_predicates_for_terms_of_interest
pheno_predicates = prepare_predicates_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
    min_n_of_patients_with_term=2,
)

from gpsea.analysis.mtc_filter import HpoMtcFilter
mtc_filter = HpoMtcFilter.default_filter(hpo, term_frequency_threshold=0.2)
mtc_correction = 'fdr_bh'
mtc_alpha = 0.05

from gpsea.analysis.pcats.stats import FisherExactTest
count_statistic = FisherExactTest()

from gpsea.analysis.pcats import HpoTermAnalysis
analysis = HpoTermAnalysis(
    count_statistic=count_statistic,
    mtc_filter=mtc_filter,
    mtc_correction=mtc_correction,
    mtc_alpha=mtc_alpha,
)

result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=gt_predicate,
    pheno_predicates=pheno_predicates,
)
result.total_tests

len(result.phenotypes)

from gpsea.view import MtcStatsViewer
mtc_viewer = MtcStatsViewer()
mtc_report = mtc_viewer.process(result)
mtc_report

from gpsea.view import summarize_hpo_analysis
summary_df = summarize_hpo_analysis(hpo, result)
summary_df

Sex of the individual                   FEMALE           MALE          \
                                         Count Percent  Count Percent   
Developmental regression [HP:0002376]    27/81     33%   8/44     18%   
Aggressive behavior [HP:0000718]         22/72     31%   5/32     16%   
Hearing impairment [HP:0000365]          19/62     31%  16/35     46%   
Chronic constipation [HP:0012450]        42/79     53%  17/40     42%   
Pes planus [HP:0001763]                  17/81     21%  13/44     30%   
Strabismus [HP:0000486]                  18/52     35%   6/24     25%   
Joint hypermobility [HP:0001382]         18/81     22%  12/44     27%   
Microcephaly [HP:0000252]                18/68     26%   7/34     21%   
Atrial septal defect [HP:0001631]        12/17     71%   9/11     82%   
Hypotonia [HP:0001252]                   33/81     41%  16/44     36%   
Absent speech [HP:0001344]                7/33     21%   3/20     15%   
Intellectual disability [HP:0001249]     73/81     90%  39/44     89%   
Large for gestational age [HP:0001520]   12/60     20%   7/31     23%   
Anxiety [HP:0000739]                     19/72     26%   7/32     22%   
Gastroesophageal reflux [HP:0002020]     20/75     27%   9/39     23%   
Autism [HP:0000717]                      39/72     54%  16/32     50%   
Global developmental delay [HP:0001263]  73/80     91%  41/44     93%   
Feeding difficulties [HP:0011968]        26/31     84%  14/16     88%   
Recurrent infections [HP:0002719]        34/36     94%  24/26     92%   
Overweight [HP:0025502]                  25/71     35%  14/38     37%   

Sex of the individual                                                 
                                        Corrected p values  p values  
Developmental regression [HP:0002376]                  1.0  0.095207  
Aggressive behavior [HP:0000718]                       1.0  0.146999  
Hearing impairment [HP:0000365]                        1.0  0.186638  
Chronic constipation [HP:0012450]                      1.0  0.332985  
Pes planus [HP:0001763]                                1.0  0.380560  
Strabismus [HP:0000486]                                1.0  0.440588  
Joint hypermobility [HP:0001382]                       1.0  0.520747  
Microcephaly [HP:0000252]                              1.0  0.628292  
Atrial septal defect [HP:0001631]                      1.0  0.668339  
Hypotonia [HP:0001252]                                 1.0  0.703179  
Absent speech [HP:0001344]                             1.0  0.724799  
Intellectual disability [HP:0001249]                   1.0  0.768708  
Large for gestational age [HP:0001520]                 1.0  0.790404  
Anxiety [HP:0000739]                                   1.0  0.806751  
Gastroesophageal reflux [HP:0002020]                   1.0  0.821421  
Autism [HP:0000717]                                    1.0  0.831770  
Global developmental delay [HP:0001263]                1.0  1.000000  
Feeding difficulties [HP:0011968]                      1.0  1.000000  
Recurrent infections [HP:0002719]                      1.0  1.000000  
Overweight [HP:0025502]                                1.0  1.000000